<h1>Modelling</h1>

In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor

In [3]:
def importData(location):
    data = pd.read_csv(location)
    return data

In [4]:
cv_2_200 = importData('../Data/cv_2_200.csv')
cv_2_200.head()

,Unnamed: 0,article_id,title,text,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,...,windows phone,windows update,windows xp,world cup,wp_scm_comment bonus,wrote blog,york city,york times,youtube channel,youtube videos
0,0,7014291,It's Time for a YouTube Competitor,It was reported last week that Yahoo is gettin...,0.640816,5.0,3.0,1.0,4.579592,5,...,0,0,0,0,0,0,0,0,0,1
1,1,7014327,The Ultimate Roundup of 2014's Biggest April F...,April Fools' marks the day when pranking goes ...,0.549618,4.0,1.0,0.0,4.636704,7,...,0,0,0,0,0,0,0,0,0,0
2,2,7014304,A Selfie and One Garish Jacket: How the Red So...,The 2013 World Series champion Boston Red Sox ...,0.577236,8.0,1.0,0.0,4.627451,7,...,0,0,0,0,0,0,0,0,0,0
3,3,7031995,African Telecom Proposes More Diverse Emoticons,Less than a week after Apple pledged to make i...,0.525896,4.0,1.0,0.0,5.083665,9,...,0,0,0,0,1,0,0,0,0,0
4,4,6541359,"Google to Launch Improved Android Camera App, ...",Android users may soon see a series of new fea...,0.588889,6.0,9.0,0.0,4.336022,7,...,0,0,0,0,0,0,0,0,0,0


In [5]:
tfidf_2_200 = importData('../Data/tfidf_2_200.csv')
tfidf_2_200.head()

,Unnamed: 0,article_id,title,text,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,...,windows phone,windows update,windows xp,world cup,wp_scm_comment bonus,wrote blog,york city,york times,youtube channel,youtube videos
0,0,7014291,It's Time for a YouTube Competitor,It was reported last week that Yahoo is gettin...,0.640816,5.0,3.0,1.0,4.579592,5,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.715661
1,1,7014327,The Ultimate Roundup of 2014's Biggest April F...,April Fools' marks the day when pranking goes ...,0.549618,4.0,1.0,0.0,4.636704,7,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,2,7014304,A Selfie and One Garish Jacket: How the Red So...,The 2013 World Series champion Boston Red Sox ...,0.577236,8.0,1.0,0.0,4.627451,7,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
3,3,7031995,African Telecom Proposes More Diverse Emoticons,Less than a week after Apple pledged to make i...,0.525896,4.0,1.0,0.0,5.083665,9,...,0.0,0.0,0.0,0.0,0.415438,0.0,0.0,0.0,0.0,0.000000
4,4,6541359,"Google to Launch Improved Android Camera App, ...",Android users may soon see a series of new fea...,0.588889,6.0,9.0,0.0,4.336022,7,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [6]:
# Function for deleting column Unnamed: 0 and also creating df which only has numeric fields
def initialProcessing(data):
    data.drop('Unnamed: 0', axis=1, inplace=True)
    df = data.drop(['article_id', 'title', 'text', 'data_channel_cleaned', 'text_prepared', 'published_date_cleaned'], axis=1)
    return df

In [7]:
cv_2_200 = initialProcessing(cv_2_200)
cv_2_200

,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,min_avg_key,max_avg_key,avg_avg_key,href_avg_shares,...,windows phone,windows update,windows xp,world cup,wp_scm_comment bonus,wrote blog,york city,york times,youtube channel,youtube videos
0,0.640816,5.0,3.0,1.0,4.579592,5,1459.500,4817.692,3370.415,1833.333,...,0,0,0,0,0,0,0,0,0,1
1,0.549618,4.0,1.0,0.0,4.636704,7,0.000,3581.094,2556.367,8500.000,...,0,0,0,0,0,0,0,0,0,0
2,0.577236,8.0,1.0,0.0,4.627451,7,2270.889,3852.800,3018.167,2076.333,...,0,0,0,0,0,0,0,0,0,0
3,0.525896,4.0,1.0,0.0,5.083665,9,0.000,3581.094,1871.403,2200.000,...,0,0,0,0,1,0,0,0,0,0
4,0.588889,6.0,9.0,0.0,4.336022,7,3264.605,8453.451,5155.323,805.000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,0.606557,6.0,2.0,0.0,4.913978,4,3264.605,5709.823,4020.608,1950.000,...,0,0,0,0,0,0,0,0,0,0
999,0.491018,4.0,2.0,0.0,4.674627,6,0.000,4352.491,2358.572,4200.000,...,0,0,0,0,0,0,0,0,0,0
1000,0.634518,3.0,0.0,2.0,4.177665,9,2126.118,6060.771,3296.052,3400.000,...,0,0,0,0,0,0,0,0,0,0
1001,0.447368,7.0,1.0,0.0,4.667961,8,0.000,4763.083,2709.129,0.000,...,0,0,0,0,0,0,0,0,0,0


In [8]:
tfidf_2_200 = initialProcessing(tfidf_2_200)
tfidf_2_200

,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,min_avg_key,max_avg_key,avg_avg_key,href_avg_shares,...,windows phone,windows update,windows xp,world cup,wp_scm_comment bonus,wrote blog,york city,york times,youtube channel,youtube videos
0,0.640816,5.0,3.0,1.0,4.579592,5,1459.500,4817.692,3370.415,1833.333,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.715661
1,0.549618,4.0,1.0,0.0,4.636704,7,0.000,3581.094,2556.367,8500.000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
2,0.577236,8.0,1.0,0.0,4.627451,7,2270.889,3852.800,3018.167,2076.333,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
3,0.525896,4.0,1.0,0.0,5.083665,9,0.000,3581.094,1871.403,2200.000,...,0.0,0.0,0.0,0.0,0.415438,0.0,0.000000,0.0,0.0,0.000000
4,0.588889,6.0,9.0,0.0,4.336022,7,3264.605,8453.451,5155.323,805.000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,0.606557,6.0,2.0,0.0,4.913978,4,3264.605,5709.823,4020.608,1950.000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
999,0.491018,4.0,2.0,0.0,4.674627,6,0.000,4352.491,2358.572,4200.000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
1000,0.634518,3.0,0.0,2.0,4.177665,9,2126.118,6060.771,3296.052,3400.000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
1001,0.447368,7.0,1.0,0.0,4.667961,8,0.000,4763.083,2709.129,0.000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000


In [9]:
tfidf_2_200.sort_values(by=['shares'], ascending=False)

,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,min_avg_key,max_avg_key,avg_avg_key,href_avg_shares,...,windows phone,windows update,windows xp,world cup,wp_scm_comment bonus,wrote blog,york city,york times,youtube channel,youtube videos
895,0.492511,33.0,0.0,1.0,5.125162,7,3264.605,7674.667,5451.520,45940.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
904,0.356121,15.0,4.0,0.0,4.431962,6,0.000,7045.000,3295.043,9871.429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
407,0.598361,6.0,1.0,1.0,4.748000,6,0.000,4715.200,2943.471,2200.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,0.452484,33.0,3.0,1.0,4.536511,4,3249.933,11625.000,5546.815,2627.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
555,0.687783,4.0,13.0,0.0,4.600000,7,0.000,8439.371,3658.861,1100.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,0.555324,3.0,1.0,0.0,4.985477,5,710.000,4360.563,3146.869,1370.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
791,0.687783,4.0,1.0,4.0,4.336634,7,2373.367,5988.238,3973.711,4200.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
754,0.647059,6.0,2.0,0.0,4.717791,8,1212.333,3830.235,2917.420,3600.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
413,0.460630,19.0,3.0,0.0,4.773216,6,0.000,5532.872,2924.013,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
np.log(1+12290)

9.416622566214981

In [11]:
type(tfidf_2_200)

pandas.core.frame.DataFrame

<h3>Splitting Data in Preparation for Modelling</h3>

In [12]:
def splitData(data):
    X = data.drop('shares',axis=1)
    y = data[['shares']]
    x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    print((x_train.shape, x_test.shape), (y_train.shape,y_test.shape))
    return [x_train, x_test, y_train, y_test]

In [13]:
def standardizeTransform(x_train, x_test, y_train, y_test):
    scaler = StandardScaler().fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    y_train_log = np.log(1+y_train)
    y_test_log = np.log(1+y_test)
    return [x_train_scaled, x_test_scaled, y_train_log, y_test_log]

In [14]:
x_train_cv, x_test_cv, y_train_cv, y_test_cv = splitData(cv_2_200)

((802, 223), (201, 223)) ((802, 1), (201, 1))


In [15]:
y_test_cv

,shares
626,2000
629,1400
847,1400
514,1100
365,1700
...,...
208,757
777,2200
334,2900
210,6500


In [16]:
x_train_scaled_cv, x_test_scaled_cv, y_train_log_cv, y_test_log_cv = standardizeTransform(x_train_cv, x_test_cv, y_train_cv, y_test_cv)

In [17]:
y_test_log_cv.sort_values(by=['shares'], ascending=False)

,shares
516,9.416623
904,9.416623
30,9.416623
394,9.416623
54,9.416623
...,...
826,6.284134
199,6.263398
816,6.190315
784,6.084499


In [18]:
y_train_log_cv.sort_values(by=['shares'], ascending=False)

,shares
610,9.416623
282,9.416623
430,9.416623
555,9.416623
352,9.416623
...,...
841,5.948035
791,5.774552
754,5.572154
413,5.056246


In [19]:
x_train_tf, x_test_tf, y_train_tf, y_test_tf = splitData(tfidf_2_200)

((802, 223), (201, 223)) ((802, 1), (201, 1))


In [20]:
x_train_scaled_tf, x_test_scaled_tf, y_train_log_tf, y_test_log_tf = standardizeTransform(x_train_tf, x_test_tf, y_train_tf, y_test_tf)

<h1>Models</h1>

In [22]:
# Fit Model and Evaluate
def fitPredict(x_train, y_train, x_test, y_test, model):
    model.fit(x_train, y_train)

    y_hat = np.exp(model.predict(x_test))

    print("Train R2 score: "+ str(model.score(x_train,y_train)))
    print("Test R2 score: "+ str(r2_score(y_test,y_hat)))
    print("Test MAE: "+ str(mean_absolute_error(y_test,y_hat)))
    print("Test RMSE: "+ str(np.sqrt(mean_squared_error(y_test,y_hat))))

    return model


In [ ]:
type(y_train_cv)

In [ ]:
# Baselining using mean

print("Test R2 score: "+ str(r2_score(y_test_cv,np.mean(y_train_cv))))
print("Test MAE: "+ str(mean_absolute_error(y_test,y_hat)))
print("Test RMSE: "+ str(np.sqrt(mean_squared_error(y_test,y_hat))))

In [35]:
# Linear Model Test set prediction and Evaluation
print('Count Vectorizer Test Set Results:')
lm = fitPredict(x_train_scaled_cv, y_train_log_cv, x_test_scaled_cv, y_test_log_cv, LinearRegression())
print('tfidf Test set Results:')
lm = fitPredict(x_train_scaled_tf, y_train_log_tf, x_test_scaled_tf, y_test_log_tf, LinearRegression())

Count Vectorizer Test Set Results:
Train R2 score: 0.35906749933010995


C:\Users\ituser\AppData\Local\Temp\ipykernel_23844\3925403474.py:5: RuntimeWarning: overflow encountered in exp
  y_hat = np.exp(model.predict(x_test))


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

Linear Model seems to make predictions that are unreasonably high causing the above error.

In [25]:
# Ridge Test set prediction and Evaluation
print('Count Vectorizer Train Set Results:')
rm = fitPredict(x_train_scaled_cv, y_train_log_cv, x_test_scaled_cv, y_test_log_cv, Ridge())
print('tfidf Train set Results:')
rm = fitPredict(x_train_scaled_tf, y_train_log_tf, x_test_scaled_tf, y_test_log_tf, Ridge())

Count Vectorizer Train Set Results:
Train R2 score: 0.3594376181280087
Test R2 score: -1208850618.2705479
Test MAE: 4948.069583587039
Test RMSE: 29386.361782110846
tfidf Train set Results:
Train R2 score: 0.35030067368941864
Test R2 score: -12195794.008421885
Test MAE: 2390.8515982752965
Test RMSE: 2951.6479731448494


TfIdf data performs significantly better than Count Vectorizer data when using the Ridge Regression Model.

In [28]:
# MLPRegressor Test set prediction and Evaluation
print('Count Vectorizer Test Set Results:')
rm = fitPredict(x_train_scaled_cv, y_train_log_cv, x_test_scaled_cv, y_test_log_cv, MLPRegressor())
print('tfidf Test set Results:')
rm = fitPredict(x_train_scaled_tf, y_train_log_tf, x_test_scaled_tf, y_test_log_tf, MLPRegressor())

Count Vectorizer Test Set Results:


C:\Users\ituser\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ituser\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ituser\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train R2 score: 0.6369392091011905
Test R2 score: -1.020786741671778e+41
Test MAE: 2.1286458306782265e+19
Test RMSE: 2.7003932432870113e+20
tfidf Test set Results:
Train R2 score: 0.6580150558740787
Test R2 score: -269769433443.1649
Test MAE: 58321.8647826916
Test RMSE: 438991.2584095346


Performance of the model is not good, but TfIdf still performs much better than Count Vectorizer Data for the MLPRegressor Model

In [29]:
# Gradient Boosting Regressor Test set prediction and Evaluation
print('Count Vectorizer Test Set Results:')
rm = fitPredict(x_train_scaled_cv, y_train_log_cv, x_test_scaled_cv, y_test_log_cv, GradientBoostingRegressor())
print('tfidf Test set Results:')
rm = fitPredict(x_train_scaled_tf, y_train_log_tf, x_test_scaled_tf, y_test_log_tf, GradientBoostingRegressor())

Count Vectorizer Test Set Results:


C:\Users\ituser\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train R2 score: 0.550353263985458
Test R2 score: -7192321.258238166
Test MAE: 2105.6486334417677
Test RMSE: 2266.7001396900177
tfidf Test set Results:


C:\Users\ituser\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train R2 score: 0.5287719271283919
Test R2 score: -6711515.379266732
Test MAE: 2081.9820575564727
Test RMSE: 2189.6254364260426


Gradient Boosting Model performs better than the other models using test data. And again, the best results come from TfIdf data.

In [32]:
# Elastic Net Test set prediction and Evaluation
print('Count Vectorizer Test Set Results:')
rm = fitPredict(x_train_scaled_cv, y_train_log_cv, x_test_scaled_cv, y_test_log_cv, ElasticNet())
print('tfidf Test set Results:')
rm = fitPredict(x_train_scaled_tf, y_train_log_tf, x_test_scaled_tf, y_test_log_tf, ElasticNet())

Count Vectorizer Test Set Results:
Train R2 score: 0.0
Test R2 score: -4918072.997380169
Test MAE: 1874.3771773979906
Test RMSE: 1874.3773679581027
tfidf Test set Results:
Train R2 score: 0.0
Test R2 score: -4918072.997380169
Test MAE: 1874.3771773979906
Test RMSE: 1874.3773679581027


In [34]:
# XG Boost Regressor Test set prediction and Evaluation
print('Count Vectorizer Test Set Results:')
rm = fitPredict(x_train_scaled_cv, y_train_log_cv, x_test_scaled_cv, y_test_log_cv, XGBRegressor())
print('tfidf Test set Results:')
rm = fitPredict(x_train_scaled_tf, y_train_log_tf, x_test_scaled_tf, y_test_log_tf, XGBRegressor())

Count Vectorizer Test Set Results:
Train R2 score: 0.996991300089808
Test R2 score: -8565580.459056232
Test MAE: 2196.8519935105073
Test RMSE: 2473.648156817093
tfidf Test set Results:
Train R2 score: 0.9872991985104576
Test R2 score: -8519095.04914936
Test MAE: 2211.0176981902337
Test RMSE: 2466.9267810078254
